# MNIST Dataset Overview

[1]:http://yann.lecun.com/exdb/mnist/

MNIST图像数据集使用形如$[28, 28]$的二阶数组来表示每张图像，数组中的每个元素对应一个像素点，该数据集中的图像都是256阶灰度图，像素值0表示白色（背景），256表示黑色（前景）。由于每张图像的尺寸都是28×28像素，为了方便连续存储，我们可以将形如$[28, 28]$的二阶数组“摊平”成形如$[784]$的一阶数组。数组中的784个元素共同组成了一个784维的向量。

<img width=80% height=80% src="imgs/02/01.jpeg" alt="imgs/02/01.jpeg" title="图1" />

More info: [http://yann.lecun.com/exdb/mnist/][1]

In [4]:
from __future__ import print_function

# import dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('datas/mnist', one_hot=True)

import tensorflow as tf

Extracting datas/mnist\train-images-idx3-ubyte.gz
Extracting datas/mnist\train-labels-idx1-ubyte.gz
Extracting datas/mnist\t10k-images-idx3-ubyte.gz
Extracting datas/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
# super params
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# neural network params
n_hidden_1 = 256 # first level nodes nums
n_hidden_2 = 256 # sencond level nodes nums
num_input = 784 # mnist datasize (28*28)
num_classes = 10 # mnist classify nums (0-9)

# input train datas
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [4]:
# weights & biases
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [5]:
# neural network
def neural_next(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    return out_layer

In [6]:
# definded models
logits = neural_next(X)

# definded lossfunction & optimizer
loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_func)

# definded accuracy
correct_pred = tf.equal(tf.arg_max(logits, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# init params
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `argmax` instead


In [10]:
# start training
with tf.Session() as sess:
    # running initializer
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # training
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step ==1:
            loss, acc = sess.run([loss_func, accuracy], feed_dict={X: batch_x, Y: batch_y})
            
            print("Step" + str(step) + \
                  " >>> Minibatch_Loss=" + "{:.4f}".format(loss) + \
                  "   Training_Accuracy=" + "{:.3f}".format(acc))
            
    print("Optimization Finished!")
            
    # calculate accuracy
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images,
                                                                     Y: mnist.test.labels}))

Step1 >>> Minibatch_Loss=7484.1797   Training_Accuracy=0.477
Step100 >>> Minibatch_Loss=296.4350   Training_Accuracy=0.820
Step200 >>> Minibatch_Loss=166.7542   Training_Accuracy=0.812
Step300 >>> Minibatch_Loss=36.3193   Training_Accuracy=0.914
Step400 >>> Minibatch_Loss=54.9062   Training_Accuracy=0.906
Step500 >>> Minibatch_Loss=34.8032   Training_Accuracy=0.867
Optimization Finished!
Testing Accuracy: 0.852
